# Annotate Q&A with MetaMap

In [1]:
%load_ext autoreload
%autoreload 2

import pickle 
from mmlrestclient import handle_request
import json
from datasets import load_dataset
import pandas as pd
from tqdm.notebook import tqdm 
import ast 
from inference import read_jsonl 

def to_pickle(df, f):
    with open(f, 'wb') as fname:
        pickle.dump(df, fname)

def open_pickle(f):
    with open(f, 'rb') as file:
        data = pickle.load(file)
    return data  

In [4]:
#train_data_list = open_pickle("annot_train_medqa_front_5.0.pkl")
train_data_list = open_pickle("new_list_data_dict.pkl")

In [2]:
node_df = pd.read_csv("/n/home01/ruthjohnson/kg_paper_revision/connected_node_v3_df.csv", sep='\t')
keep_cui = set(node_df.loc[node_df['ntype'] == 'UMLS_CUI']['node_id'].str.split(':', expand=True)[0].tolist())
keep_cui = keep_cui - set(['C1457887', 'C0011900', 'C0087111', 'C0013227', 'C1533734', 'C0012634', 
                           'C0543467', 'C1533734', 'C0030705', 'C0031831', 'C0031809', 'C1522326',
                           'C0025080', 'C0242821', 'C0040223', 'C0376558', 'C1704338', 'C0086418'])

## MedQA (TRAIN)

In [2]:
train_medqa_list = read_jsonl("/n/home01/ruthjohnson/ruthjohnson/kg_paper_revision/medqa/MMedBench/Train/English.jsonl",
                            is_with_rationale=False)

False


In [3]:
#
# BACK (original, limit types, no thresh)
#
node_df = pd.read_csv("/n/home01/ruthjohnson/kg_paper_revision/connected_node_v3_df.csv", sep='\t')
keep_cui = set(node_df.loc[node_df['ntype'] == 'UMLS_CUI']['node_id'].str.split(':', expand=True)[0].tolist())

annot_train_medqa_list = []
metamap_thresh = 0.0

for orig_entry in tqdm(train_medqa_list[7500: ]):
        annot_entry = orig_entry.copy()
        qa_str = orig_entry['input']
        params = [('inputtext', qa_str),
                ('docformat', 'freetext'),
                        ('resultformat', 'mmi'), 
                        ('sourceString', 'ICD9CM'), ('sourceString', 'ICD10CM'), ('sourceString', 'ATC'), ('sourceString', 'RXNORM'),
                        ('sourceString', 'LNC'), ('sourceString', 'SNOMEDCT_US'), ('sourceString', 'CPT'),
                        ('semanticTypeString', 'bpoc'),('semanticTypeString', 'blor'),('semanticTypeString', 'bsoj'), 
                        ('semanticTypeString', 'neop'),('semanticTypeString', 'diap'),('semanticTypeString', 'dsyn'),
                        ('semanticTypeString', 'moft'),('semanticTypeString', 'topp'),('semanticTypeString', 'celc'),
                        ('semanticTypeString', 'aapp'),('semanticTypeString', 'horm'),('semanticTypeString', 'phsu'),
                        ('semanticTypeString', 'sosy'), ('semanticTypeString', 'lbpr'),('semanticTypeString', 'cell'),
                        ('semanticTypeString', 'orch'),('semanticTypeString', 'gngm'),('semanticTypeString', 'patf')
                        ]

        resp = handle_request('https://ii.nlm.nih.gov/metamaplite/rest/annotate',
                               'text/plain',
                               params)

        text_list = resp.text.split('\n')
        text_list = [x for x in text_list if x]
        text_list = [x for x in text_list if 'MMI' in x]
        if len(text_list) > 0:
                cui_list = set([x.split('|')[4] for x in text_list if ((x.split('|')[1]=='MMI') & (float(x.split('|')[2]) > metamap_thresh))])
                cui_list = list(cui_list & keep_cui)
        else:
                cui_list = []

        cui_token_list= [('<%s>' % x) for x in cui_list]
        annot_entry['input'] = annot_entry['input'] + ' ' + ' '.join(cui_token_list) 
        annot_train_medqa_list.append(annot_entry)

  0%|          | 0/2678 [00:00<?, ?it/s]

In [4]:
#to_pickle(annot_train_medqa_list, "annot_train_medqa_orig_1.pkl".format(metamap_thresh)) #7500 
to_pickle(annot_train_medqa_list, "annot_train_medqa_orig_2.pkl".format(metamap_thresh)) #7500 

In [6]:
annot_train_medqa_list_1 = open_pickle("annot_train_medqa_orig_1.pkl")
annot_train_medqa_list_2 = open_pickle("annot_train_medqa_orig_2.pkl")

In [7]:
annot_train_medqa_list = annot_train_medqa_list_1 + annot_train_medqa_list_2

In [8]:
to_pickle(annot_train_medqa_list, "annot_train_medqa_orig.pkl")

In [4]:
#
# BACK
#
annot_train_medqa_list = []
metamap_thresh = 5.0

for orig_entry in tqdm(train_medqa_list):
        annot_entry = orig_entry.copy()
        qa_str = orig_entry['input']
        params = [('inputtext', qa_str),
                ('docformat', 'freetext'),
                        ('resultformat', 'mmi'), 
                        ('sourceString', 'ICD9CM'), ('sourceString', 'ICD10CM'), ('sourceString', 'ATC'), ('sourceString', 'RXNORM'),
                        ('sourceString', 'LNC'), ('sourceString', 'SNOMEDCT_US'), ('sourceString', 'CPT'),
                        ('semanticTypeString', 'bpoc'),('semanticTypeString', 'blor'),('semanticTypeString', 'bsoj'), 
                        ('semanticTypeString', 'neop'),('semanticTypeString', 'diap'),('semanticTypeString', 'dsyn'),
                        ('semanticTypeString', 'moft'),('semanticTypeString', 'topp'),('semanticTypeString', 'celc'),
                        ('semanticTypeString', 'aapp'),('semanticTypeString', 'horm'),('semanticTypeString', 'phsu'),
                        ('semanticTypeString', 'sosy'), ('semanticTypeString', 'lbpr'),('semanticTypeString', 'cell'),
                        ('semanticTypeString', 'orch'),('semanticTypeString', 'gngm'),('semanticTypeString', 'patf'),
                        ('semanticTypeString', 'all')
                        ]

        resp = handle_request('https://ii.nlm.nih.gov/metamaplite/rest/annotate',
                               'text/plain',
                               params)

        text_list = resp.text.split('\n')
        text_list = [x for x in text_list if x]
        text_list = [x for x in text_list if 'MMI' in x]
        if len(text_list) > 0:
                cui_list = set([x.split('|')[4] for x in text_list if ((x.split('|')[1]=='MMI') & (float(x.split('|')[2]) > metamap_thresh))])
                cui_list = list(cui_list & keep_cui)
        else:
                cui_list = []

        cui_token_list= [('<%s>' % x) for x in cui_list]
        annot_entry['input'] = annot_entry['input'] + ' ' + ' '.join(cui_token_list) 
        annot_train_medqa_list.append(annot_entry)

  0%|          | 0/10178 [00:00<?, ?it/s]

In [5]:
to_pickle(annot_train_medqa_list, "annot_train_medqa_back_{}.pkl".format(metamap_thresh))

In [ ]:
#
# FRONT
#
annot_train_medqa_list = []
metamap_thresh = 5.0

for orig_entry in tqdm(train_medqa_list):
        annot_entry = orig_entry.copy()
        qa_str = orig_entry['input']
        params = [('inputtext', qa_str),
                ('docformat', 'freetext'),
                        ('resultformat', 'mmi'), 
                        ('sourceString', 'ICD9CM'), ('sourceString', 'ICD10CM'), ('sourceString', 'ATC'), ('sourceString', 'RXNORM'),
                        ('sourceString', 'LNC'), ('sourceString', 'SNOMEDCT_US'), ('sourceString', 'CPT'),
                        ('semanticTypeString', 'bpoc'),('semanticTypeString', 'blor'),('semanticTypeString', 'bsoj'), 
                        ('semanticTypeString', 'neop'),('semanticTypeString', 'diap'),('semanticTypeString', 'dsyn'),
                        ('semanticTypeString', 'moft'),('semanticTypeString', 'topp'),('semanticTypeString', 'celc'),
                        ('semanticTypeString', 'aapp'),('semanticTypeString', 'horm'),('semanticTypeString', 'phsu'),
                        ('semanticTypeString', 'sosy'), ('semanticTypeString', 'lbpr'),('semanticTypeString', 'cell'),
                        ('semanticTypeString', 'orch'),('semanticTypeString', 'gngm'),('semanticTypeString', 'patf'),
                        ('semanticTypeString', 'all')
                        ]

        resp = handle_request('https://ii.nlm.nih.gov/metamaplite/rest/annotate',
                               'text/plain',
                               params)

        text_list = resp.text.split('\n')
        text_list = [x for x in text_list if x]
        text_list = [x for x in text_list if 'MMI' in x]
        if len(text_list) > 0:
                cui_list = set([x.split('|')[4] for x in text_list if ((x.split('|')[1]=='MMI') & (float(x.split('|')[2]) > metamap_thresh))])
                cui_list = list(cui_list & keep_cui)
        else:
                cui_list = []

        cui_token_list= [('<%s>' % x) for x in cui_list]
        annot_entry['input'] = ' '.join(cui_token_list) + annot_entry['input']
        annot_train_medqa_list.append(annot_entry)

  0%|          | 0/10178 [00:00<?, ?it/s]

In [5]:
to_pickle(annot_train_medqa_list, "annot_train_medqa_front_{}.pkl".format(metamap_thresh))

In [3]:
#
# Extract just the CUI tokens
#
annot_train_medqa_list = open_pickle("annot_train_medqa_orig.pkl")

In [4]:
import re 
cui_token_list = []
for x in annot_train_medqa_list:
    input_string = x['input']
    tokens = re.findall(r'<(C[^>]*)>', input_string)
    cui_token_list.append(tokens)

In [9]:
len(annot_train_medqa_list)

10179

In [8]:
to_pickle(cui_token_list, "tokens_train_medqa_orig.pkl")

## MedQA (Test)

In [6]:
test_medqa_list = read_jsonl("/n/home01/ruthjohnson/ruthjohnson/kg_paper_revision/medqa/MMedBench/Test/English.jsonl",
                            is_with_rationale=False)

False


In [7]:
annot_train_medqa_list = []
metamap_thresh = 5.0

for orig_entry in tqdm(test_medqa_list):
        annot_entry = orig_entry.copy()
        qa_str = orig_entry['input']
        params = [('inputtext', qa_str),
                ('docformat', 'freetext'),
                        ('resultformat', 'mmi'), 
                        ('sourceString', 'ICD9CM'), ('sourceString', 'ICD10CM'), ('sourceString', 'ATC'), ('sourceString', 'RXNORM'),
                        ('sourceString', 'LNC'), ('sourceString', 'SNOMEDCT_US'), ('sourceString', 'CPT'),
                        ('semanticTypeString', 'bpoc'),('semanticTypeString', 'blor'),('semanticTypeString', 'bsoj'), 
                        ('semanticTypeString', 'neop'),('semanticTypeString', 'diap'),('semanticTypeString', 'dsyn'),
                        ('semanticTypeString', 'moft'),('semanticTypeString', 'topp'),('semanticTypeString', 'celc'),
                        ('semanticTypeString', 'aapp'),('semanticTypeString', 'horm'),('semanticTypeString', 'phsu'),
                        ('semanticTypeString', 'sosy'), ('semanticTypeString', 'lbpr'),('semanticTypeString', 'cell'),
                        ('semanticTypeString', 'orch'),('semanticTypeString', 'gngm'),('semanticTypeString', 'patf'),
                        ('semanticTypeString', 'all')
                        ]

        resp = handle_request('https://ii.nlm.nih.gov/metamaplite/rest/annotate',
                               'text/plain',
                               params)

        text_list = resp.text.split('\n')
        text_list = [x for x in text_list if x]
        text_list = [x for x in text_list if 'MMI' in x]
        if len(text_list) > 0:
                cui_list = set([x.split('|')[4] for x in text_list if ((x.split('|')[1]=='MMI') & (float(x.split('|')[2]) > metamap_thresh))])
                cui_list = list(cui_list & keep_cui)
        else:
                cui_list = []

        cui_token_list= [('<%s>' % x) for x in cui_list]
        annot_entry['input'] = ' '.join(cui_token_list) + annot_entry['input']
        annot_train_medqa_list.append(annot_entry)

  0%|          | 0/1273 [00:00<?, ?it/s]

In [8]:
to_pickle(annot_train_medqa_list, "annot_test_medqa_front_{}.pkl".format(metamap_thresh))

## AfriMed (Test)

In [2]:
afrimedqa = load_dataset("intronhealth/afrimedqa_v2")
df = pd.DataFrame(afrimedqa['train'])
qa_df = df.loc[df['question_type'] == 'mcq'][['question_clean', 'answer_options', 'correct_answer']]

In [ ]:
# limit to 4 options

In [ ]:


response_map = {'option1': 'A', 'option2': 'B', 'option3': 'C', 'option4': 'D', 'option5': 'E'}
instruction_template = new_list_data_dict[0]['instruction']

af_list_data_dict = []

for i, row in qa_df.iterrows():
    question = row['question_clean']
    answer_options = ast.literal_eval(row['answer_options'])
    correct_response = row['correct_answer']

    format_question = '###Question: {} ###Options: A. {} B. {} C. {} D. {} E. {}'.format(
        question, 
        answer_options['option1'], 
        answer_options['option2'], 
        answer_options['option3'], 
        answer_options['option4'], 
        answer_options['option5'])

    format_answer = '###Answer: OPTION {} IS CORRECT.'.format(response_map[correct_response]) 

    format_dict = {'instruction': instruction_template,
    'input': format_question,
    'output': format_answer}
    
    af_list_data_dict.append(format_dict)


In [6]:
df.loc[(df['question_type'] == 'mcq')].groupby('tier').count()

,sample_id,split,gender,discipline,clinical_experience,country,question_type,prompt,question,question_clean,...,neg_percent,quality,rated_african,rated_correct,rated_omission,rated_hallucination,rated_reasonable,rated_harmful,rated_bias,version
tier,,,,,,,,,,,,,,,,,,,,,
crowdsourced,129,129,129,129,129,129,129,0,129,129,...,129,13,129,13,13,13,13,13,13,129
expert,3910,3910,3910,3910,3910,3910,3910,0,3910,3910,...,3910,1,3910,1,1,1,1,1,1,3910


In [5]:
df.loc[(df['question_type'] == 'mcq')].groupby('specialty').count()

,sample_id,split,gender,discipline,clinical_experience,country,question_type,prompt,question,question_clean,...,neg_percent,quality,rated_african,rated_correct,rated_omission,rated_hallucination,rated_reasonable,rated_harmful,rated_bias,version
specialty,,,,,,,,,,,,,,,,,,,,,
Allergy_and_Immunology,5,5,5,5,5,5,5,0,5,5,...,5,0,5,0,0,0,0,0,0,5
Anesthesiology,12,12,12,12,12,12,12,0,12,12,...,12,0,12,0,0,0,0,0,0,12
Cardiology,175,175,175,175,175,175,175,0,175,175,...,175,0,175,0,0,0,0,0,0,175
Dermatology,36,36,36,36,36,36,36,0,36,36,...,36,1,36,1,1,1,1,1,1,36
Emergency_Medicine,27,27,27,27,27,27,27,0,27,27,...,27,0,27,0,0,0,0,0,0,27
Endocrinology,142,142,142,142,142,142,142,0,142,142,...,142,0,142,0,0,0,0,0,0,142
Family_Medicine,9,9,9,9,9,9,9,0,9,9,...,9,1,9,1,1,1,1,1,1,9
Gastroenterology,134,134,134,134,134,134,134,0,134,134,...,134,2,134,2,2,2,2,2,2,134
General_Surgery,498,498,498,498,498,498,498,0,498,498,...,498,0,498,0,0,0,0,0,0,498


In [3]:
set(afrimedqa['train']['clinical_experience'])

{'Consultant',
 'Intern',
 'Licensed Professional',
 None,
 'Other',
 'Researcher',
 'Resident',
 'Scientist',
 'Student',
 'Technician'}

In [10]:
new_list_data_dict = open_pickle("afr_list_data_dict.pkl")

In [12]:
medqa_test_list = open_pickle("test_new_list_data_dict.pkl")

In [13]:
medqa_test_list

[{'instruction': "You're a English doctor, kindly address the medical queries according to the patient's account. Answer with the best option directly.",
  'input': '###Question: A junior orthopaedic surgery resident is completing a carpal tunnel repair with the department chairman as the attending physician. During the case, the resident inadvertently cuts a flexor tendon. The tendon is repaired without complication. The attending tells the resident that the patient will do fine, and there is no need to report this minor complication that will not harm the patient, as he does not want to make the patient worry unnecessarily. He tells the resident to leave this complication out of the operative report. Which of the following is the correct next action for the resident to take? Which of the following is the best treatment for this patient? ###Options: A. Disclose the error to the patient and put it in the operative report B. Tell the attending that he cannot fail to disclose this mistak

In [11]:
new_list_data_dict

[{'instruction': "You're a English doctor, kindly address the medical queries according to the patient's account. Answer with the best option directly.",
  'input': "###Question: Which of the following conditions is characterized by an abnormal enlargement of the thyroid gland, often resulting in a visibly enlarged neck, and is endemic in certain regions of Africa due to dietary iodine deficiency? ###Options: A. Hashimoto's thyroiditis B. Graves' disease C. Toxic multinodular goiter D. Simple (nontoxic) goiter E. Thyroid cancer<C0040132> <C0018021> <C3159206> <C0027530>",
  'output': '###Answer: OPTION D IS CORRECT.'},
 {'instruction': "You're a English doctor, kindly address the medical queries according to the patient's account. Answer with the best option directly.",
  'input': '###Question: Which of the following infectious diseases, endemic to certain regions of Africa, is caused by the bacterium Mycobacterium leprae and primarily affects the skin, nerves, and mucous membranes? ##

In [7]:
qa_df = qa_df.loc[~qa_df['correct_answer'].str.contains(',')]

In [ ]:


to_pickle(new_af_list_data_dict, "afr_list_data_dict.pkl")

In [11]:
from transformers import HfArgumentParser
from transformers.training_args import TrainingArguments
from dataclasses import dataclass, field
from train import ModelArguments, DataArguments
import transformers 

# Define simulated command-line arguments
args_list = [
    "--model_name_or_path", "Henrychur/MMed-Llama-3-8B-EnIns",
    "--data_path", "/n/holylfs06/LABS/mzitnik_lab/Lab/ruthjohnson/kg_paper_revision/medqa/MMedBench/Train_eng",
    "--output_dir", "/n/home01/ruthjohnson/ruthjohnson/kg_paper_revision/medqa/ft_plus_linear_results",    
    "--bf16", "True",
    "--num_train_epochs", "3",
    "--per_device_train_batch_size", "1",
    "--gradient_accumulation_steps", "32",
    "--evaluation_strategy", "no",
    "--save_strategy", "epoch",
    "--save_total_limit", "3",
    "--learning_rate", "2e-5",
    "--weight_decay", "0.",
    "--warmup_ratio", "0.03",
    "--lr_scheduler_type", "cosine",
    "--logging_steps", "1",
    "--is_lora", "True",
    "--local_rank", "16",
    "--target_modules", "q_proj", "v_proj", "proj_layer"
]

# Initialize the HfArgumentParser
parser = transformers.HfArgumentParser((ModelArguments, DataArguments, TrainingArguments))

# Parse the simulated arguments into dataclasses
model_args, data_args, training_args = parser.parse_args_into_dataclasses(args_list)

/n/home01/ruthjohnson/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/n/home01/ruthjohnson/.local/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/n/home01/ruthjohnson/.local/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 t

In [12]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_args.model_name_or_path,
    model_max_length=2048,
    use_fast=False,
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token

In [13]:
from train_custom_model import make_supervised_data_module

data_module = make_supervised_data_module(tokenizer=tokenizer, data_args=data_args)

In [ ]:
from tqdm.notebook import tqdm 

new_list_data_dict = []

for x in tqdm(list_data_dict):
        qa_str = x['input'].split("###Options") # just do whole thing?
        params = [('inputtext', qa_str),
                ('docformat', 'freetext'),
                        ('resultformat', 'mmi'), 
                        ('sourceString', 'ICD9CM'), ('sourceString', 'ICD10CM'), ('sourceString', 'ATC'), ('sourceString', 'RXNORM'),
                        ('sourceString', 'LNC'), ('sourceString', 'SNOMEDCT_US'), ('sourceString', 'CPT'),
                        #('semanticTypeString', 'all')
                        ('semanticTypeString', 'bpoc'),
                        ('semanticTypeString', 'blor'),
                        ('semanticTypeString', 'bsoj'),
                        ('semanticTypeString', 'neop'),
                        ('semanticTypeString', 'diap'),
                        ('semanticTypeString', 'dsyn'),
                        ('semanticTypeString', 'moft'),
                        ('semanticTypeString', 'topp'),
                        ('semanticTypeString', 'celc'),
                        ('semanticTypeString', 'aapp'),
                        ('semanticTypeString', 'horm'),
                        ('semanticTypeString', 'phsu'),
                        ('semanticTypeString', 'sosy'),
                        ('semanticTypeString', 'lbpr'),
                        ('semanticTypeString', 'cell'),
                        ('semanticTypeString', 'orch'),
                        ('semanticTypeString', 'gngm'),
                        ('semanticTypeString', 'patf'),
                        ]

        resp = handle_request(url, acceptfmt, params)
        text_list = resp.text.split('\n')
        text_list = [x for x in text_list if x]
        text_list = [x for x in text_list if '|' in x]
        if len(text_list) > 0:
                cui_list = set([x.split('|')[4] for x in text_list if ((x.split('|')[1]=='MMI'))]) # & (float(x.split('|')[2]) > 0)
                cui_list = list(cui_list & keep_cui)
        else:
                cui_list = []

        cui_token_list= [('<%s>' % x) for x in cui_list]
        x['input'] += ' '.join(cui_token_list)
        new_list_data_dict.append(x)

  0%|          | 0/1273 [00:00<?, ?it/s]

In [ ]:
import pickle 

def to_pickle(df, f):
    with open(f, 'wb') as fname:
        pickle.dump(df, fname)

def open_pickle(f):
    with open(f, 'rb') as file:
        data = pickle.load(file)
    return data

#to_pickle(new_list_data_dict, "new_list_data_dict.pkl")
#to_pickle(new_list_data_dict, "test_new_list_data_dict.pkl")
to_pickle(new_list_data_dict, "test_list_data_dict.pkl")

In [10]:
new_list_data_dict[10]

{'instruction': "You're a English doctor, kindly address the medical queries according to the patient's account. Answer with the best option directly.",
 'input': '###Question: A 46-year-old woman comes to the physician because of a 2-week history of diplopia and ocular pain when reading the newspaper. She also has a 3-month history of amenorrhea, hot flashes, and increased sweating. She reports that she has been overweight all her adult life and is happy to have lost 6.8-kg (15-lb) of weight in the past 2 months. Her pulse is 110/min, and blood pressure is 148/98 mm Hg. Physical examination shows moist palms and a nontender thyroid gland that is enlarged to two times its normal size. Ophthalmologic examination shows prominence of the globes of the eyes, bilateral lid retraction, conjunctival injection, and an inability to converge the eyes. There is no pain on movement of the extraocular muscles. Visual acuity is 20/20 bilaterally. Neurologic examination shows a fine resting tremor of

In [69]:
print(qa_str)

print(node_df.loc[node_df['node_id'].str.split(':', expand=True)[0].isin(cui_list)][['node_id', 'node_name']])

###Question: A 68-year-old man is referred to the cardiology department with complaints of shortness of breath on exertion that has been progressive for the last 6 months. He has a history of diabetes mellitus type II which is controlled with diet alone. He has a temperature of 37.1℃ (98.8℉), the pulse is 76/min, and the blood pressure is 132/86 mm Hg. Physical examination is notable for a systolic murmur heard best at the right upper sternal border with radiation to the carotid arteries. ECG shows left ventricular hypertrophy and absent Q waves. Transthoracic echocardiogram shows an elevated aortic pressure gradient with severe leaflet calcification and left ventricular diastolic dysfunction. Which of the following has a survival benefit for this patient’s cardiac problem? Which of the following is the best treatment for this patient? 
                 node_id                node_name
74833  C0003483:umls_cui                    Aorta
74963  C0007272:umls_cui         Carotid arteries
7